## IMPORTING DATA

In [1]:
import pyspark
from pyspark import SparkConf
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql.functions import *

In [2]:
conf = SparkConf()
conf = conf.setAppName("PySpark LSH") \
    .set("spark.local.dir", "C:/tmp") \
    .set("spark.executor.instances", "2") \
#     .set("spark.mongodb.input.uri", "mongodb://127.0.0.1:27017/bda.lsh") \
#     .set("spark.mongodb.output.uri", "mongodb://127.0.0.1:27017/bda.lsh") \
#     .set('spark.jars.packages','org.mongodb.spark:mongo-spark-connector_2.12-3.0.1') \


In [3]:
sc.stop()
sc = SparkContext(conf=conf)

In [4]:
spark = SparkSession(sc)

df = spark.read.csv('test.csv')

# df = spark.read.format('com.mongodb.spark.sql.DefaultSource').load()
df.show()


+---+--------------------+
|_c0|                 _c1|
+---+--------------------+
|  0|Man’s Search For ...|
|  1|Walden by Henry D...|
|  2|The Art Of Thinki...|
|  3|Thinking Fast and...|
|  4|The Willpower Ins...|
|  5|Flow by Mihaly Cs...|
|  6|The Story of the ...|
|  7| Spark by John Ratey|
|  8|Sapiens by Yuval ...|
|  9|Invisible Man by ...|
| 10|Influence by Robe...|
| 11| Quiet by Susan Cain|
| 12|When I Stop Talki...|
| 13|The Greatest Sale...|
| 14|   Paradox of Choice|
| 15|  The Power Of Habit|
| 16|       Daily rituals|
| 17|      Getting To Yes|
| 18|The Autobiography...|
| 19|    The Moral Animal|
+---+--------------------+
only showing top 20 rows



## PREPROCESSING

In [5]:
# q1 = df.select(col("qid1").alias("qid"), col("question1").alias("question"))
# q2 = df.select(col("qid2").alias("qid"), col("question2").alias("question"))

In [6]:
# ques = q1.union(q2).dropDuplicates()

In [7]:
# ques = ques.orderBy('qid', ascending=True).limit(20).cache()

In [8]:
ques = df.select(col("_c0").alias("qid"), col("_c1").alias("question")).cache()
ques.show()

+---+--------------------+
|qid|            question|
+---+--------------------+
|  0|Man’s Search For ...|
|  1|Walden by Henry D...|
|  2|The Art Of Thinki...|
|  3|Thinking Fast and...|
|  4|The Willpower Ins...|
|  5|Flow by Mihaly Cs...|
|  6|The Story of the ...|
|  7| Spark by John Ratey|
|  8|Sapiens by Yuval ...|
|  9|Invisible Man by ...|
| 10|Influence by Robe...|
| 11| Quiet by Susan Cain|
| 12|When I Stop Talki...|
| 13|The Greatest Sale...|
| 14|   Paradox of Choice|
| 15|  The Power Of Habit|
| 16|       Daily rituals|
| 17|      Getting To Yes|
| 18|The Autobiography...|
| 19|    The Moral Animal|
+---+--------------------+
only showing top 20 rows



In [9]:
qid_to_ques = dict()

for row in ques.collect():
    qid_to_ques[row.qid] = row.question

## GENERATING SHINGLES

In [10]:
def ques_to_shingle(q, k=5):
    
    n = len(q)
    
    shingle_set = []
    
    for i in range(n-k+1):
        shingle_set.append(q[i:i+k])
    
    return shingle_set

In [11]:
create_shingles = udf(ques_to_shingle, ArrayType(StringType()))
ques_shingle = ques.withColumn("shingles",create_shingles(ques.question)).cache()
ques_shingle.show()

+---+--------------------+--------------------+
|qid|            question|            shingles|
+---+--------------------+--------------------+
|  0|Man’s Search For ...|[Man’s, an’s , n’...|
|  1|Walden by Henry D...|[Walde, alden, ld...|
|  2|The Art Of Thinki...|[The A, he Ar, e ...|
|  3|Thinking Fast and...|[Think, hinki, in...|
|  4|The Willpower Ins...|[The W, he Wi, e ...|
|  5|Flow by Mihaly Cs...|[Flow , low b, ow...|
|  6|The Story of the ...|[The S, he St, e ...|
|  7| Spark by John Ratey|[Spark, park , ar...|
|  8|Sapiens by Yuval ...|[Sapie, apien, pi...|
|  9|Invisible Man by ...|[Invis, nvisi, vi...|
| 10|Influence by Robe...|[Influ, nflue, fl...|
| 11| Quiet by Susan Cain|[Quiet, uiet , ie...|
| 12|When I Stop Talki...|[When , hen I, en...|
| 13|The Greatest Sale...|[The G, he Gr, e ...|
| 14|   Paradox of Choice|[Parad, arado, ra...|
| 15|  The Power Of Habit|[The P, he Po, e ...|
| 16|       Daily rituals|[Daily, aily , il...|
| 17|      Getting To Yes|[Getti, ettin,

## MAPPING SET-OF-SHINGLES TO SET-OF-INTEGERS

In [12]:
from pyspark.ml.feature import CountVectorizer
cv = CountVectorizer(inputCol="shingles", outputCol="hashed_shingles1",minDF=1,vocabSize=1<<20)

model = cv.fit(ques_shingle)

ques_shingle_hashed = model.transform(ques_shingle).cache()
ques_shingle_hashed.select(["qid","shingles","hashed_shingles1"]).show(truncate=False)


+---+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|qid|shingles                                                                                                                                                                                                                                                        

In [13]:
hashed_shingles = udf( (lambda shingle : shingle.indices.tolist()) , ArrayType(IntegerType()))
ques_shingle_hashed = ques_shingle_hashed.withColumn("hashed_shingles",hashed_shingles(ques_shingle_hashed.hashed_shingles1)).cache()
ques_shingle_hashed.show()

+---+--------------------+--------------------+--------------------+--------------------+
|qid|            question|            shingles|    hashed_shingles1|     hashed_shingles|
+---+--------------------+--------------------+--------------------+--------------------+
|  0|Man’s Search For ...|[Man’s, an’s , n’...|(514,[29,32,62,76...|[29, 32, 62, 76, ...|
|  1|Walden by Henry D...|[Walde, alden, ld...|(514,[9,30,64,85,...|[9, 30, 64, 85, 8...|
|  2|The Art Of Thinki...|[The A, he Ar, e ...|(514,[0,1,2,6,8,1...|[0, 1, 2, 6, 8, 1...|
|  3|Thinking Fast and...|[Think, hinki, in...|(514,[0,4,8,10,11...|[0, 4, 8, 10, 11,...|
|  4|The Willpower Ins...|[The W, he Wi, e ...|(514,[20,23,25,37...|[20, 23, 25, 37, ...|
|  5|Flow by Mihaly Cs...|[Flow , low b, ow...|(514,[5,14,15,21,...|[5, 14, 15, 21, 9...|
|  6|The Story of the ...|[The S, he St, e ...|(514,[4,7,11,12,1...|[4, 7, 11, 12, 13...|
|  7| Spark by John Ratey|[Spark, park , ar...|(514,[27,45,67,71...|[27, 45, 67, 71, ...|
|  8|Sapie

In [14]:
ques_shingle_hashed = ques_shingle_hashed.select(["qid","hashed_shingles"]).cache()
ques_shingle_hashed.show()

+---+--------------------+
|qid|     hashed_shingles|
+---+--------------------+
|  0|[29, 32, 62, 76, ...|
|  1|[9, 30, 64, 85, 8...|
|  2|[0, 1, 2, 6, 8, 1...|
|  3|[0, 4, 8, 10, 11,...|
|  4|[20, 23, 25, 37, ...|
|  5|[5, 14, 15, 21, 9...|
|  6|[4, 7, 11, 12, 13...|
|  7|[27, 45, 67, 71, ...|
|  8|[55, 59, 65, 83, ...|
|  9|[1, 9, 36, 129, 1...|
| 10|[1, 2, 34, 51, 11...|
| 11|[3, 20, 63, 88, 1...|
| 12|[0, 41, 68, 77, 7...|
| 13|[25, 39, 43, 82, ...|
| 14|[38, 90, 101, 142...|
| 15|[23, 28, 54, 114,...|
| 16|[72, 94, 147, 314...|
| 17|[93, 134, 166, 18...|
| 18|[6, 7, 33, 40, 46...|
| 19|[50, 104, 182, 18...|
+---+--------------------+
only showing top 20 rows



## MIN-HASHING TO GENERATE SIGNATURES

In [15]:
num_hash_fns = 20

In [16]:
def hash_func(a,x,n):
    # (a*x+1)%n - family of hash functions
    return (a*x+1)%n

In [17]:
def min_hash(shingle, n=165931):
    
    signature = [float("inf")]*num_hash_fns
    
    for h in range(num_hash_fns):
        for i in shingle:
            hash_val = hash_func(h+1,i,n)
            if hash_val < signature[h]:
                signature[h] = hash_val
    
    return signature

In [18]:
create_signatures = udf(min_hash, ArrayType(IntegerType()))
ques_signatures = ques_shingle_hashed.withColumn("signatures",create_signatures(ques_shingle_hashed.hashed_shingles)).cache()
ques_signatures.show()

+---+--------------------+--------------------+
|qid|     hashed_shingles|          signatures|
+---+--------------------+--------------------+
|  0|[29, 32, 62, 76, ...|[30, 59, 88, 117,...|
|  1|[9, 30, 64, 85, 8...|[10, 19, 28, 37, ...|
|  2|[0, 1, 2, 6, 8, 1...|[1, 1, 1, 1, 1, 1...|
|  3|[0, 4, 8, 10, 11,...|[1, 1, 1, 1, 1, 1...|
|  4|[20, 23, 25, 37, ...|[21, 41, 61, 81, ...|
|  5|[5, 14, 15, 21, 9...|[6, 11, 16, 21, 2...|
|  6|[4, 7, 11, 12, 13...|[5, 9, 13, 17, 21...|
|  7|[27, 45, 67, 71, ...|[28, 55, 82, 109,...|
|  8|[55, 59, 65, 83, ...|[56, 111, 166, 22...|
|  9|[1, 9, 36, 129, 1...|[2, 3, 4, 5, 6, 7...|
| 10|[1, 2, 34, 51, 11...|[2, 3, 4, 5, 6, 7...|
| 11|[3, 20, 63, 88, 1...|[4, 7, 10, 13, 16...|
| 12|[0, 41, 68, 77, 7...|[1, 1, 1, 1, 1, 1...|
| 13|[25, 39, 43, 82, ...|[26, 51, 76, 101,...|
| 14|[38, 90, 101, 142...|[39, 77, 115, 153...|
| 15|[23, 28, 54, 114,...|[24, 47, 70, 93, ...|
| 16|[72, 94, 147, 314...|[73, 145, 217, 28...|
| 17|[93, 134, 166, 18...|[94, 187, 280,

## DIVIDING INTO BANDS

In [19]:
bands = 5
rows_per_band = num_hash_fns//bands

def LSH_buckets(signature, k = 1001):
    
    hash_buckets = []
    
    for b in range(bands):
        sign_b = 1
        for i in range(b*rows_per_band,(b+1)*rows_per_band):
            if (signature[i] is not None):
                sign_b *= (signature[i]%k)
        
        hash_buckets.append(sign_b%k)
    
    return hash_buckets

In [20]:
create_hash_buckets = udf(LSH_buckets, StructType([ StructField("band"+str(b+1), IntegerType(), False) for b in range(bands)]))
ques_bands = ques_signatures.withColumn("Hash_Bands",create_hash_buckets(ques_signatures.signatures)).cache()
ques_bands.show()

+---+--------------------+--------------------+--------------------+
|qid|     hashed_shingles|          signatures|          Hash_Bands|
+---+--------------------+--------------------+--------------------+
|  0|[29, 32, 62, 76, ...|[30, 59, 88, 117,...|[715, 371, 360, 4...|
|  1|[9, 30, 64, 85, 8...|[10, 19, 28, 37, ...|[644, 352, 546, 6...|
|  2|[0, 1, 2, 6, 8, 1...|[1, 1, 1, 1, 1, 1...|     [1, 1, 1, 1, 1]|
|  3|[0, 4, 8, 10, 11,...|[1, 1, 1, 1, 1, 1...|     [1, 1, 1, 1, 1]|
|  4|[20, 23, 25, 37, ...|[21, 41, 61, 81, ...|[952, 770, 689, 5...|
|  5|[5, 14, 15, 21, 9...|[6, 11, 16, 21, 2...|[154, 468, 931, 1...|
|  6|[4, 7, 11, 12, 13...|[5, 9, 13, 17, 21...|[936, 924, 644, 2...|
|  7|[27, 45, 67, 71, ...|[28, 55, 82, 109,...|[770, 567, 689, 1...|
|  8|[55, 59, 65, 83, ...|[56, 111, 166, 22...|[364, 826, 815, 8...|
|  9|[1, 9, 36, 129, 1...|[2, 3, 4, 5, 6, 7...|[120, 21, 143, 63...|
| 10|[1, 2, 34, 51, 11...|[2, 3, 4, 5, 6, 7...|[120, 21, 143, 63...|
| 11|[3, 20, 63, 88, 1...|[4, 7, 1

In [21]:
# ques_band_buckets = ques_bands.select(["qid","question","signatures"] + ["Hash_Bands.band"+str(b+1) for b in range(bands)] ).cache()
# ques_band_buckets.show()

## GENERATING CANDIDATE PAIRS

In [22]:
def comparison(qid, hashed_shingles, thresh=0.75):
    n = len(hashed_shingles)
    
    qid_all = []

    for i in range(n):
        for j in range(i+1,n):
            q1 = set(hashed_shingles[i])
            q2 = set(hashed_shingles[j])
            
            jac = len(q1&q2)/(1+len(q1|q2))
            if jac>=thresh:
            qid1 = qid[i]
            qid2 = qid[j]
            qid_all.append([qid1,qid2])

    if len(qid_all)==0:
        return None
    else:
        return qid_all

In [25]:
import time
import itertools

all_candidates = []

for b in range(1,bands+1):
    start = time.time()
    ques_band_b = ques_bands.groupby("Hash_Bands.band"+str(b)).agg(collect_list("qid"),collect_list("hashed_shingles")).cache()
    ques_band_b.show()
    
#     candidate_pairs = udf(lambda x:list(itertools.combinations(x, 2)), ArrayType(ArrayType(IntegerType())))
#     candidates_band_b = ques_band_b.withColumn("Candidates",candidate_pairs(ques_band_b[1])).cache()
    candidate_pairs = udf(comparison, ArrayType(ArrayType(IntegerType())))
    candidates_band_b = ques_band_b.withColumn("Candidates",candidate_pairs(ques_band_b[1],ques_band_b[2])).cache()

    candidates_band_b.show()
    
    pdf = candidates_band_b.select("Candidates").cache()
    pdf.show()
    
    pdf1 = pdf.select(explode(pdf.Candidates).alias("Candidates")).cache()
    pdf1.show()
    
    all_candidates.append(pdf1)
    print(f"{b} done. Time:{time.time()-start}s")
    

+-----+-----------------+-----------------------------+
|band1|collect_list(qid)|collect_list(hashed_shingles)|
+-----+-----------------+-----------------------------+
|    1|       [2, 3, 12]|         [[0, 1, 2, 6, 8, ...|
|  770|              [7]|         [[27, 45, 67, 71,...|
|  715|              [0]|         [[29, 32, 62, 76,...|
|  182|             [18]|         [[6, 7, 33, 40, 4...|
|  308|             [17]|         [[93, 134, 166, 1...|
|  120|          [9, 10]|         [[1, 9, 36, 129, ...|
|  644|              [1]|         [[9, 30, 64, 85, ...|
|  936|              [6]|         [[4, 7, 11, 12, 1...|
|  154|              [5]|         [[5, 14, 15, 21, ...|
|  677|             [20]|            [[193, 384, 488]]|
|  637|         [11, 22]|         [[3, 20, 63, 88, ...|
|  952|          [4, 16]|         [[20, 23, 25, 37,...|
|  364|              [8]|         [[55, 59, 65, 83,...|
|  945|             [15]|         [[23, 28, 54, 114...|
|  287|             [21]|         [[26, 47, 75, 

+---------------+
|     Candidates|
+---------------+
|             []|
|             []|
|[[,], [,], [,]]|
|          [[,]]|
|             []|
|             []|
|             []|
|          [[,]]|
|          [[,]]|
|             []|
|             []|
|             []|
|             []|
|             []|
|          [[,]]|
|          [[,]]|
+---------------+

+----------+
|Candidates|
+----------+
|       [,]|
|       [,]|
|       [,]|
|       [,]|
|       [,]|
|       [,]|
|       [,]|
|       [,]|
+----------+

3 done. Time:127.13912916183472s
+-----+-----------------+-----------------------------+
|band4|collect_list(qid)|collect_list(hashed_shingles)|
+-----+-----------------+-----------------------------+
|  300|             [16]|         [[72, 94, 147, 31...|
|   91|             [18]|         [[6, 7, 33, 40, 4...|
|  209|             [17]|         [[93, 134, 166, 1...|
|    1|       [2, 3, 12]|         [[0, 1, 2, 6, 8, ...|
|  692|              [1]|         [[9, 30, 64, 85, ...|
|

In [26]:
final_cands = all_candidates[0]
for i in range(1,bands):
    final_cands = final_cands.union(all_candidates[i])
final_cands.dropDuplicates().show()

+----------+
|Candidates|
+----------+
|       [,]|
+----------+

